# Main script to Join Data on Instrument 2 Fire Location

Modules: N/A <br>
Author: Jordan Meyer <br>
Email: jordan.meyer@berkeley.edu <br>
Date created: Feb 18, 2023 <br>

**Citations (data sources)**


**Citations (persons)**
1. Cornelia Ilin 

**Preferred environment**
1. Code written in Jupyter Notebooks

In [1]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install geopandas --quiet
    from google.colab import drive

    drive.mount("/content/drive")
    in_dir = (
        in_instrument
    ) = "/content/drive/MyDrive/capstone_fire/notebooks/instrument_2/"
    import drive.MyDrive.capstone_fire.modules.deep_ols as deep_ols
else:
    in_dir = in_instrument = "../data/instrument_2/"
    %cd '..'
    import modules.deep_ols as deep_ols
    %cd 'notebooks'

/Users/jordan/Documents/GitHub/fire_capstone
/Users/jordan/Documents/GitHub/fire_capstone/notebooks


In [2]:
import calendar
import os
from datetime import date, timedelta

# geography
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import shapely

# Moved from sklearn.neighbors to sklearn.metrics following their package change
import sklearn.metrics
from shapely.geometry import Point

# import cartopy.crs as ccrs
# import contextily as ctx
# import fiona
# import netCDF4 as ncdf
# import osmnx as ox
# from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
# from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


dist = sklearn.metrics.DistanceMetric.get_metric("haversine")

# ignore warnings
import warnings

from tqdm.notebook import tqdm, trange

warnings.filterwarnings("ignore")

# START HERE FOR AGGREGATION

## Aggregation of the dataframe for OLS

In [3]:
instrument_df = pd.read_csv(in_instrument + "aggregated_draft_2.csv",)
instrument_df

,ZCTA,year_month,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_bins_acres,ins_6_bins_raw,zip_pm25,zip_elevation,elevation_difference,zip_wspd
0,89010,199101,0.017147,2.996985,2.193307,2.237374e-03,2.237374e-03,24.261588,12.450976,4784.0,-3523.444444,0.878208
1,89010,199102,0.004105,2.134088,0.522705,1.519277e-03,1.519277e-03,16.875655,11.255854,4784.0,-3412.000000,0.283772
2,89010,199104,0.041240,7.240880,3.803710,4.474625e-03,4.474625e-03,51.209927,7.899268,4784.0,-3777.333333,1.547416
3,89010,199106,0.002230,0.048681,0.002707,3.951177e-07,3.951177e-07,0.096979,8.160244,4784.0,-3290.000000,0.893248
4,89010,199107,0.010883,0.936064,1.136627,5.962933e-04,5.962933e-04,6.661194,9.101220,4784.0,-3369.000000,0.817715
...,...,...,...,...,...,...,...,...,...,...,...,...
212285,97635,201608,0.126739,608.902926,11.801323,2.240041e-01,2.240041e-01,2586.061987,3.558125,7370.0,-6782.840909,0.782387
212286,97635,201609,0.095820,488.387292,9.758271,1.940952e-01,1.940952e-01,2186.942079,1.619554,7370.0,-6757.846154,0.464931
212287,97635,201610,0.053249,188.397190,6.272099,6.879567e-02,6.879567e-02,836.545477,2.673929,7370.0,-6184.090909,2.635240
212288,97635,201611,0.010921,10.890098,1.231006,9.119611e-03,9.119611e-03,96.892259,3.386071,7370.0,-6325.900000,1.906592


In [4]:
demographics_df = pd.read_csv(in_instrument+'demographics.csv')

In [5]:
demographics_df['year']=demographics_df['year'].map(lambda x: str(x)[-4:])
demographics_df = demographics_df.rename(columns={'zip_code':'ZCTA'})

In [6]:
temp = instrument_df.reset_index()
instrument_df['year'] = temp['year_month'].apply(lambda x:str(x)[:4])

In [7]:
final_df = instrument_df.merge(demographics_df, on=['year','ZCTA'], how='left')

In [8]:
final_df.to_csv(in_instrument + 'finalish_df_2.csv')

1778